# Understanding the AWRA Community Modelling System package and its modules

   1. AWRA-CMS package and modules
   2. AWRA CMS concepts and classes  


## 1. AWRA-CMS package and modules

#### There are 7 module components to the AWRAMS python package
* ***awrams.models***        : the AWRA-L model code
* ***awrams.simulation***    : tools to run the model 
    * **ondemand**: keeps the inputs and outputs stored in memory - for fast interactive use
    * **server**: direct to disk output, used for large scale runs (larger than available memory)  
* ***awrams.visualisation*** : contains functions to facilitate viewing of outputs and inputs across periods and spatial extents
* ***awrams.calibration***   : contains tools for calibration and evaluation of sensitivity to model parameters
* ***awrams.benchmarking***  : contains tools to allow comparison of multiple model outputs to observations
* ***awrams.utils***         : contains various support tools used throughout the rest of the system
* ***awrams.cluster***       : contains support classes for clustered (MPI) runs

#### There is a copy of the code provided in the awrams_cm folder to aid understanding
     
   - [Models folder]       :   awrams/models
   - [Simulation folder]   :   awrams/simulation
   - [Visualisation folder]:   awrams/visualisation
   - [Calibration folder]  :   awrams/calibration
   - [Benchmarking folder] :   awrams/benchmarking 
   - [Cluster folder] :   awrams/cluster  

[Models folder]: /tree/packages/awrams/models
[Simulation folder]: /tree/packages/awrams/simulation/
[Visualisation folder]: /tree/packages/awrams/visualisation
[Calibration folder]: /tree/packages/awrams/calibration
[Benchmarking folder]: /tree/packages/awrams/benchmarking
[Cluster folder]: /tree/packages/awrams/cluster/

# 2. AWRA CMS concepts and classes 



## 2.1 Configuring AWRA simulations and calibrations

 - Setting up a simulation or calibration run starts by defining:

   - A.  The **model profile** using the config manager
   - B.  The inputs and outputs via **node mappings**
      1. Inputs:
         - the forcing inputs (rain, temperature, radiation) 
         - the spatial inputs (grids of various landscape properties)
         - model parameters (calibrated or fixed)
      2. Outputs:
         - model outputs, either in memory or written to storage
         
   - C.  The spatial **extent** 
   - D.  The modelling time **period**



## 2.1.A. Get a model profile using the config manager

In [ ]:
### Load a model profile from the config_manager
from awrams.utils import config_manager

model_profile = config_manager.get_model_profile('awral','v6_default')

In [ ]:
# Get the model settings; this is everything that is configurable in a model
model_settings = model_profile.get_settings()

model_settings

## 2.1.B. Input/Output Mappings and Nodegraphs

 - ***Mappings*** are Python dictionaries used to describe the data-flow
      - AWRAMS ***NodeGraph*** subsystem (awrams.utils.nodegraph) transforms this **configuration** info into runnable code.
 - Input mapping: the ‘data’ portion of a model; everything that happens before the core computational loop
      - loading of files
      - infilling
      - unit conversion etc. 
 - Example:
     - AWRA-L model uses a single temperature input, but AWAP supplies two (minimum and maximum temperature).  
     - The default AWRA-L input mapping loads the AWAP inputs then rectifies and computes a weighted average of these, before passing the single value on to the core model code. 


### Need to map values to these AWRA-L input parameters

- Various [nodes.py] types are used to map input values to these parameters
    - climate input from netcdf files:  nodes.forcing_from_ncfiles(CLIMATE_DATA,v[0],v[1])
    - static spatial inputs from grids: nodes.spatial_from_file(SPATIAL_FILE,'parameters/%s' % grid)
    - assigning a constant value eg. air pressure mapping['pair'] = nodes.const(97500.)
    - transformations: eg.
         - average temperature mapping['tat'] = nodes.mix('tmin','tmax',0.75)
         - multiplying a spatial grid by an input parameter mapping['s0max'] = nodes.mul('s0max_scale','s0fracawc_grid',100.)


[nodes.py]: /tree/packages/awrams/utils/nodegraph/nodes.py

In [ ]:
# View the default configuration mapping
input_mapping = model_profile.get_input_mapping()

In [ ]:
input_mapping

## 2.1.C. ***Extent*** specification

Gridded spatial domains

In [ ]:
# load extents
from awrams.utils import extents

In [ ]:
# default extent - all of Australia
def_ext = extents.get_default_extent()  ## Australia, set as reference extent
def_ext

In [ ]:
#### Any rectangular extent
tas_extent = def_ext.icoords[-39.5:-44, 143.5: 149]   # Tasmania
tas_extent

In [ ]:
#### a single point based on [lat, lon] pair
cell_extent  = def_ext.icoords[-34,117]
print(cell_extent)
print(cell_extent.cell_count)

## 2.1.D. ***Period*** specification

Time periods - convenience functions that create a Pandas DatetimeIndex

In [ ]:
from awrams.utils import datetools as dt

In [ ]:
period = dt.dates('2000', '2010')

In [ ]:
period

In [ ]:
period = dt.dates('dec 2010 - 31 jan 2011')

In [ ]:
period